<a href="https://colab.research.google.com/github/AlexLiang230410/nem-battery-arbitrage/blob/main/notebooks/02_threshold_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: imports and base configuration (same parameters as baseline)

from battery_core import (
    simulate_arbitrage,
    summarize_performance,
    grid_search_thresholds,
    adaptive_threshold_search,
)
from data_utils import load_price_range

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # for heatmap

# User-configurable parameters (copied from baseline notebook)
region = "VIC1"
start_year = 2024
start_month = 12
end_year = 2025
end_month = 11
dt_mins = 5
power_mw = 10
energy_mwh = 20
charge_threshold = 30      # baseline values (for later comparison)
discharge_threshold = 70
efficiency = 0.85
visualization_year = 2025
visualization_month = 11

data_dir = "data/aemo_price_and_demand"


In [ ]:
# Cell 2: load price data for the search period

price_df = load_price_range(
    region=region,
    start_year=start_year,
    start_month=start_month,
    end_year=end_year,
    end_month=end_month,
    data_dir=data_dir,
)

print("Loaded price data:")
print("  shape:", price_df.shape)
print("  index range:", price_df.index.min(), "→", price_df.index.max())
display(price_df.head())
display(price_df.tail())


In [ ]:
# Cell 3: define search ranges for charge and discharge thresholds

charge_threshold_range = np.arange(-100, 101, 20)   # from -100 to +100, step 20
discharge_threshold_range = np.arange(20, 201, 20)  # from 20 to 200, step 20

print("Charge threshold candidates:", charge_threshold_range[:5], "...", charge_threshold_range[-5:])
print("Discharge threshold candidates:", discharge_threshold_range[:5], "...", discharge_threshold_range[-5:])
print("Total combinations (before filtering ct < dt):",
      len(charge_threshold_range) * len(discharge_threshold_range))


In [ ]:
n_charge = len(charge_threshold_range)
n_discharge = len(discharge_threshold_range)

print("n_charge =", n_charge)
print("n_discharge =", n_discharge)
print("naive combinations =", n_charge * n_discharge)


In [ ]:
# Cell 4: run grid search over threshold combinations

grid_search_df, best_params = adaptive_threshold_search(
    price_df=price_df,
    dt_mins=dt_mins,
    power_mw=power_mw,
    energy_mwh=energy_mwh,
    charge_threshold_range=charge_threshold_range,
    discharge_threshold_range=discharge_threshold_range,
    efficiency=efficiency,
    show_progress=True,
)

print("Grid search completed.")
print("Number of evaluated combinations:", len(grid_search_df))
display(grid_search_df.head())


In [ ]:
# Cell 5: inspect best-performing thresholds and compare with baseline

print("Best parameters found by grid search:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# Compute baseline performance (using your original thresholds)
baseline_result = simulate_arbitrage(
    price_df=price_df,
    dt_mins=dt_mins,
    power_mw=power_mw,
    energy_mwh=energy_mwh,
    charge_threshold=charge_threshold,
    discharge_threshold=discharge_threshold,
    efficiency=efficiency,
)
baseline_summary = summarize_performance(
    result_df=baseline_result,
    dt_mins=dt_mins,
    power_mw=power_mw,
    energy_mwh=energy_mwh,
    print_summary=False,
)

# Compute optimal performance
optimal_result = simulate_arbitrage(
    price_df=price_df,
    dt_mins=dt_mins,
    power_mw=power_mw,
    energy_mwh=energy_mwh,
    charge_threshold=best_params["charge_threshold"],
    discharge_threshold=best_params["discharge_threshold"],
    efficiency=efficiency,
)
optimal_summary = summarize_performance(
    result_df=optimal_result,
    dt_mins=dt_mins,
    power_mw=power_mw,
    energy_mwh=energy_mwh,
    print_summary=False,
)

print("\n=== Revenue comparison (baseline vs optimal) ===")
print(f"Baseline thresholds:  charge={charge_threshold}, discharge={discharge_threshold}")
print(f"  Baseline total revenue (AUD): {baseline_summary['total_revenue']:.2f}")

print(f"\nOptimal thresholds:   charge={best_params['charge_threshold']}, "
      f"discharge={best_params['discharge_threshold']}")
print(f"  Optimal total revenue (AUD):  {optimal_summary['total_revenue']:.2f}")


In [ ]:
# Cell 6: visualise total revenue as a heatmap over thresholds

# Pivot to 2D matrix: rows = charge_threshold, columns = discharge_threshold
pivot_df = grid_search_df.pivot(
    index="charge_threshold",
    columns="discharge_threshold",
    values="total_revenue",
)

plt.figure(figsize=(10, 6))
sns.heatmap(
    pivot_df,
    cmap="viridis",  # default colormap
    cbar_kws={"label": "Total revenue (AUD)"},
)

plt.title("Total revenue across charge / discharge thresholds")
plt.xlabel("Discharge threshold ($/MWh)")
plt.ylabel("Charge threshold ($/MWh)")

plt.tight_layout()
plt.show()


In [ ]:
# Cell 7 (optional): inspect time series under optimal thresholds

vis_start = pd.Timestamp(
    year=visualization_year,
    month=visualization_month,
    day=1,
)
vis_end = vis_start + pd.DateOffset(months=1)

optimal_window = optimal_result.loc[vis_start:vis_end].copy()

print(f"Optimal scenario window: {vis_start} → {vis_end}")
print("Window shape:", optimal_window.shape)
display(optimal_window)
